In [1]:
import pandas as pd

Q2

In [2]:
flights = pd.read_csv('flights.csv')
flights.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,CRS_ARR_TIME,ARR_TIME,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 16
0,2019-01-01,9E,MSP,MKE,1122,1155.0,1251,1259.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,9E,ATL,PHF,954,1002.0,1137,1132.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,9E,PHF,ATL,1213,1207.0,1410,1408.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,9E,MSP,CLE,1005,1015.0,1302,1257.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,9E,MSP,RDU,1540,1535.0,1921,1854.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


**1**. Find the month and day in which the longest arrival delay occurred.

In [7]:
flights.loc[flights['CARRIER_DELAY'].idxmax(), 'FL_DATE']
#flights.iloc[flights['ArrivalDelay'].argmax(), [0, 1]]

'2019-01-05'

**2**. Find the three airlines with the longest arrival delays.

In [9]:
flights.loc[flights['CARRIER_DELAY'].nlargest(3).index, 'OP_CARRIER']
#flights.sort_values('ArrivalDelay', ascending=False)['Airline'][:3]
#flights.groupby('Airline')['ArrivalDelay'].max().sort_values(ascending=False)[:3]
#flights.groupby('OP_CARRIER')['CARRIER_DELAY'].max()

85404     AA
294548    AA
373872    YV
Name: OP_CARRIER, dtype: object

**3**. Find the three airlines with the highest number of cancelled or diverted flights.

In [29]:
flights[(flights['Cancelled'] == 1) | (flights['Diverted'] == 1)]['Airline'].value_counts()[:3]

#flights['DivertedOrCancelled'] = flights['Diverted'] + flights['Cancelled']
#flights.groupby('Airline')['DivertedOrCancelled'].sum().nlargest(3)

AA    180
OO    163
EV    161
Name: Airline, dtype: int64

**4**. Find the airlines with more than 2% of cancelled flights. For each such airline, print its name
and the precentage of cancelled flights.

In [39]:
#flights.groupby('Airline').filter(lambda x: x['Cancelled'].mean() > 0.02) \
#       .groupby('Airline')['Cancelled'].mean()
pct_cancelled = flights.groupby('Airline')['Cancelled'].apply(lambda x: x.sum() / len(x))
pct_cancelled[pct_cancelled > 0.02]

Airline
EV    0.024923
MQ    0.043791
OO    0.021554
Name: Cancelled, dtype: float64

**5**. For each origin and destination, find the average and variance of the airtime.

In [16]:
flights.groupby(['Origin', 'Dest'])['AirTime'].agg(['mean', 'var']).head()

mean        var
Origin Dest                       
ATL    ABE    96.387097  45.778495
       ABQ   170.500000  87.866667
       ABY    28.578947   6.590643
       ACY    91.333333  11.466667
       AEX    78.725000  47.332692

**6**. Find the airline that makes the most flights between 500 and 1,000 miles.

In [17]:
flights[flights['Distance'].between(500, 1000)].groupby('Airline').size().idxmax()

'DL'

**7**. Create a bar plot showing the number of flights originating from each city.

In [18]:
df = flights.groupby('Origin').size()
df.plot.bar();

**8**. Change the Weekday column to have the day name instead of day number, e.g. 1 - Sun, 2 - Mon, etc.

In [19]:
week_day = { 1: 'SUN', 2: 'Mon', 3: 'Tue', 4: 'Wed', 5: 'Thu', 6: 'Fri', 7: 'Sat'}
flights['Weekday'] = flights['Weekday'].map(week_day)
#flights['Weekday'] = flights['Weekday'].apply(lambda x: week_day[x])
flights.head()

,Month,Day,Weekday,Airline,Origin,Dest,AirTime,Distance,ArrivalDelay,Diverted,Cancelled
0,1,1,Wed,WN,LAX,SLC,94.0,590,65.0,0,0
1,1,1,Wed,UA,DEN,IAD,154.0,1452,-13.0,0,0
2,1,1,Wed,MQ,DFW,VPS,85.0,641,35.0,0,0
3,1,1,Wed,AA,DFW,DCA,126.0,1192,-7.0,0,0
4,1,1,Wed,WN,LAX,MCI,166.0,1363,39.0,0,0


**9**. Join the Month and Day columns into a single column named Date. For example, if month=3, day=15, the date column should have 3/15.

In [27]:
#flights['Date'] = flights['Month'].astype(str).str.cat(flights['Day'].astype(str), sep='/')
#flights.drop(['Month', 'Day'], axis=1, inplace=True)
flights['Date'] = flights['Month'].map(str) + '/' + flights['Day'].map(str)
flights.head()

,Month,Day,Weekday,Airline,Origin,Dest,AirTime,Distance,ArrivalDelay,Diverted,Cancelled,Date
0,1,1,4,WN,LAX,SLC,94.0,590,65.0,0,0,1/1
1,1,1,4,UA,DEN,IAD,154.0,1452,-13.0,0,0,1/1
2,1,1,4,MQ,DFW,VPS,85.0,641,35.0,0,0,1/1
3,1,1,4,AA,DFW,DCA,126.0,1192,-7.0,0,0,1/1
4,1,1,4,WN,LAX,MCI,166.0,1363,39.0,0,0,1/1


**10**. Find the longest sequence of on-time flights per airline (an on-time flight is a flight with less than 15 minutes arrival delay).

In [21]:
def max_streak(s):
    return (s >= 15).cumsum().value_counts().max() - 1
flights.groupby('Airline')['ArrivalDelay'].agg(max_streak)

Airline
AA    50.0
AS    48.0
B6    25.0
DL    91.0
EV    42.0
F9    22.0
HA    17.0
MQ    51.0
NK    24.0
OO    36.0
UA    46.0
US    44.0
VX    38.0
WN    43.0
Name: ArrivalDelay, dtype: float64

In [22]:
# flights['OnTime'] = (flights['ArrivalDelay'] < 15) * 1

# def max_streak(s):
#    s1 = s.cumsum()
#    return s.mul(s1).diff().where(lambda x: x < 0) \
#           .ffill().add(s1, fill_value=0).max()
    
# flights.groupby('Airline')['OnTime'].agg(max_streak)